## Aufgaben 3

### (a) Datensatz einlesen
 Schreiben Sie Python-Code, mit dem Sie Ihren Datensatz einlesen können. Ihre erste Aufgabe ist es, die Daten in ein entsprechendes Datenformat zu transformieren, um es später vom Dashboard zugreifbar zu machen. Sie können die Daten entweder online zugreifen oder aber die Daten lokal auf Ihrem Rechner speichern und dort dann einlesen. Sind die Daten zu gross, dann lesen Sie nur einen Teil der Daten ein.



In [4]:
import pandas as pd

In [7]:
df = pd.read_csv("Dataset_videogames sales.csv", sep=';')

In [8]:
df.head()

,Rank,Name,Platform,Platform Company,type of console,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,Nintendo,TV,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,Nintendo,TV,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,Nintendo,TV,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,Nintendo,TV,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,Nintendo,Portable,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Rank              16598 non-null  int64  
 1   Name              16598 non-null  object 
 2   Platform          16572 non-null  object 
 3   Platform Company  16572 non-null  object 
 4   type of console   16572 non-null  object 
 5   Year              16301 non-null  float64
 6   Genre             16572 non-null  object 
 7   Publisher         16514 non-null  object 
 8   NA_Sales          16572 non-null  float64
 9   EU_Sales          16572 non-null  float64
 10  JP_Sales          16572 non-null  float64
 11  Other_Sales       16572 non-null  float64
 12  Global_Sales      16572 non-null  float64
dtypes: float64(6), int64(1), object(6)
memory usage: 1.6+ MB


### (b) Filterfunktion

Implementieren Sie schon eine Filterfunktion, mit der man die Daten auch nur teilweise bezüglich eines oder mehrerer Attribute/Eigenschaften nutzen kann. (4 P.) Für welche Filteroptionen haben Sie sich entschieden? (1 P.)


## Aufgabe 4 – Diagramme und Visualisierungen mit Plotly


### (a) Diagramm
Erzeugen Sie aus Ihrem Datensatz aus Aufgabe 3 ein Diagramm mit Plotly, das Sie in Ihrem Dashboard verwenden möchten. (5 P.) Interpretieren Sie das Diagramm, indem Sie Bezug auf die dargestellten Daten nehmen. (5 P.) Erzeugen Sie einen Screenshot des Streudiagrammes und fügen Sie diesen in das Lösungs-pdf ein.


### (b) Variabel Varianten
In diesem Aufgabenteil sollen Sie das Diagramm aus (a) so erweitern, dass es Variationen in den visuellen Variablen erlaubt, also etwa eine andere Farbe, andere Formen, ein anderes Layout oder aber textuelle Ergänzungen. Erzeugen Sie einen Screenshot des Streudiagrammes und fügen Sie diesen in das Lösungs-pdf ein. (5 P.)